# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
Задание 1.
Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.
Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>
Задание 2.
Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.
Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.
ПРИМЕЧАНИЕ

Домашнее задание сдается ссылкой на репозиторий GitHub. Не сможем проверить или помочь, если вы пришлете:

    файлы;
    архивы;
    скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.
Как правильно задавать вопросы аспирантам, преподавателям и коллегам?

Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:
1) Что я делаю?
2) Какого результата я ожидаю?
3) Как фактический результат отличается от ожидаемого?
4) Что я уже попробовал сделать, чтобы исправить проблему?

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.


# Задание 1. Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>. Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи> 

In [1]:
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd

In [2]:
KEYWORDS = ['python', 'доступ', 'контент', 'трафик']

In [3]:
def get_habr_links(keywords):
    reg = 'https://habr.com/ru/all/'
    res = requests.get(reg)
    soup = BeautifulSoup(res.text, 'html.parser')
    article = soup.find_all('a', class_='post__title_link')
    articles_title_link = list(map(lambda x: x.get('href'), article))
    articles_title = list(map(lambda x: x.text, article))
    article_date = list(map(lambda x: x.text, soup.find_all('span', class_='post__time')))
    list_preview = list(map(lambda x: x.text, soup.find_all('div', class_='post__text-html')))
    article_dates = []
    for data in article_date:
        if data.startswith("сегодня"):
            article_dates.append(dt.datetime.today().date().strftime('%Y-%m-%d'))
        elif data.startswith("вчера"):
            article_dates.append((dt.datetime.today()-dt.timedelta(days=1)).date().strftime('%Y-%m-%d'))
        else:
            article_dates.append(data)

    indices = set()
    for keyword in keywords:
        for i,text in enumerate(zip(articles_title,list_preview)):
                if keyword in text[0] or keyword in text[1]:
                    indices.add(i)          
    indices = list(indices)
    habr_news = pd.DataFrame()
    for index in indices:
        row = {'date': article_dates[index], 'title': articles_title[index], 'link': articles_title_link[index]}
        habr_news = pd.concat([habr_news, pd.DataFrame([row])])
    return habr_news    

In [4]:
get_habr_links(KEYWORDS)

,date,title,link
0,2020-08-20,Кастовая дискриминация в Кремниевой долине,https://habr.com/ru/post/515968/


# Задание 2.
Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.
Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.


In [7]:
def check_avast(emails):
    url = "https://digibody.avast.com/v1/web/leaks"
    avast_leaks = pd.DataFrame()
    for email in emails:
        post_result = requests.post(url, json={"email" : email})
        for leak in post_result.json()['value']:
            row = {'email': email,
                   'date': leak['leak_info']['date'],
                   'title': leak['leak_info']['title'], 
                   'description': leak['leak_info']['description']}
            avast_leaks = pd.concat([avast_leaks, pd.DataFrame([row])])
    return avast_leaks

In [8]:
emails = ['example@example.com', 'text@yandex.ru', 'admin@yandex.ru', 'admin@google.com']
df = check_avast(emails)
df

,email,date,title,description
0,text@yandex.ru,1513900800000,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
0,text@yandex.ru,1558569600000,LiveJournal,"In 2017, social network LiveJournal's database..."
0,text@yandex.ru,1535068800000,Jadid List from Pemiblanc.com,This combolist was compiled from a variety of ...
0,text@yandex.ru,1549411200000,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
0,text@yandex.ru,1507507200000,Exploit.in combolist,The proliferation of stolen or leaked database...
...,...,...,...,...
0,admin@google.com,1477440000000,Sensitive Source,This source has been marked as sensitive due t...
0,admin@google.com,1477353600000,Tumblr,"In May 2016, microblogging platform Tumblr rev..."
0,admin@google.com,1495756800000,Leet.cc,"In February 2016, LEET's user database was all..."
0,admin@google.com,1551312000000,500px,"In July 2018, 500px's database was allegedly b..."
